In [5]:
import pandas as pd
import numpy as np

# Load the downloaded Daymet data
climate_data = pd.read_csv('/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/ClimateData/weather_data_Aqua.csv', on_bad_lines='skip')

# Check if 'Date' column exists
if 'Date' in climate_data.columns:
    # Convert 'Date' column to datetime
    climate_data['Date'] = pd.to_datetime(climate_data['Date'], errors='coerce')

    # Remove rows with missing Date values
    climate_data = climate_data.dropna(subset=['Date'])

    # Extract Day, Month, and Year from the Date column
    climate_data['Day'] = climate_data['Date'].dt.day
    climate_data['Month'] = climate_data['Date'].dt.month
    climate_data['Year'] = climate_data['Date'].dt.year

    # Calculate necessary climate variables
    T_max = climate_data['MaxTemp']  # Maximum temperature (°C)
    T_min = climate_data['MinTemp']  # Minimum temperature (°C)
    precipitation = climate_data['Precipitation']  # Precipitation (mm)
    solar_radiation = climate_data['R_n']  # Solar radiation (MJ/m²/day)
    e_a = climate_data['e_a']  # actual vapor pressure (kpa)

    # Calculate mean temperature
    T_mean = (T_max + T_min) / 2

    # Calculate saturation vapor pressure (e_s) in kPa
    e_s = 0.6108 * np.exp((17.27 * T_mean) / (T_mean + 237.3))

    # Assuming relative humidity (RH) is available or use an average value
    #RH = 62  # Example average RH of 50%
    #e_a = (RH / 100) * e_s  # Actual vapor pressure in kPa

    # Define other constants
    R_n = solar_radiation  # Net radiation (MJ/m²/day)
    G = 0  # Soil heat flux density (MJ/m²/day), often negligible
    gamma = 0.066  # Psychrometric constant (kPa/°C)
    u = 1.2  # Average wind speed in m/s (if available, else use an estimated value)

    # Calculate slope of the saturation vapor pressure curve (Δ)
    delta = (4098 * e_s) / ((T_mean + 237.3) ** 2)

    # Calculate reference ET (ET₀) using the Penman-Monteith equation
    ReferenceET = (0.408 * delta * (R_n - G) + gamma * (900 / (T_mean + 273)) * u * (e_s - e_a)) / (delta + gamma * (1 + 0.34 * u))

    # Add ET₀ to the DataFrame
    climate_data['ReferenceET'] = ReferenceET

    # Reorder the columns to bring Day, Month, Year first
    climate_data = climate_data[['Day', 'Month', 'Year'] + [col for col in climate_data.columns if col not in ['Day', 'Month', 'Year']]]

    # Save the DataFrame to a text file (CSV format)
    climate_data.to_csv('daymet_data_with_et0.csv', index=False)  # Save as a tab-separated text file

    print("Reference ET₀ calculated and saved to 'daymet_data_with_et0.csv'.")
else:
    print("The 'Date' column does not exist in the dataset.")

Reference ET₀ calculated and saved to 'daymet_data_with_et0.csv'.


In [6]:
import os
os.environ['DEVELOPMENT'] = 'True'
!pip install numpy==1.22.4
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
from aquacrop.utils import prepare_weather, get_filepath


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
###2018-RAIN-FED##018-RAIN-FED###018-RAIN-FED###018-RAIN-FED###018-RAIN-FED###018-RAIN-FED

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2018/01/01'
sim_end = '2018/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2018"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    #irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatRainfed/wheat_rainfed_2018.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [8]:
###2019-RAIN-FED##2019-RAIN-FED###2019-RAIN-FED###2019-RAIN-FED###2019-RAIN-FED###2019-RAIN-FED

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2019/01/01'
sim_end = '2019/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2019"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    #irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatRainfed/wheat_rainfed_2019.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [9]:
###2020-RAIN-FED##2020-RAIN-FED###2020-RAIN-FED###2020-RAIN-FED###2020-RAIN-FED###2020-RAIN-FED

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2020/01/01'
sim_end = '2020/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2020"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    #irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatRainfed/wheat_rainfed_2020.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [10]:
###2021-RAIN-FED##2021-RAIN-FED###2021-RAIN-FED###2021-RAIN-FED###2021-RAIN-FED###2021-RAIN-FED

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2021/01/01'
sim_end = '2021/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2021"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    #irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatRainfed/wheat_rainfed_2021.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [11]:
###2022-RAIN-FED##2022-RAIN-FED###2022-RAIN-FED###2022-RAIN-FED###2022-RAIN-FED###2022-RAIN-FED

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2022/01/01'
sim_end = '2022/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2022"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    #irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatRainfed/wheat_rainfed_2022.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [12]:
###2023-RAIN-FED##2023-RAIN-FED###2023-RAIN-FED###2023-RAIN-FED###2023-RAIN-FED###2023-RAIN-FED

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2023/01/01'
sim_end = '2023/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2023"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    #irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatRainfed/wheat_rainfed_2023.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [13]:
###2018-NET_IRRIGATION##2018-NET_IRRIGATION###2018-NET_IRRIGATION###2018-NET_IRRIGATION###2018-NET_IRRIGATION###2018-NET_IRRIGATION

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2018/01/01'
sim_end = '2018/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2018"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    #irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatNetIrriDeamad/wheat_netirridemand_2018.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [14]:
###2019-NET_IRRIGATION##2019-NET_IRRIGATION###2019-NET_IRRIGATION###2019-NET_IRRIGATION###2019-NET_IRRIGATION###2019-NET_IRRIGATION

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2019/01/01'
sim_end = '2019/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2019"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    #irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatNetIrriDeamad/wheat_netirridemand_2019.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [15]:
###2020-NET_IRRIGATION##2020-NET_IRRIGATION###2020-NET_IRRIGATION###2020-NET_IRRIGATION###2020-NET_IRRIGATION###2020-NET_IRRIGATION

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2020/01/01'
sim_end = '2020/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2020"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    #irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatNetIrriDeamad/wheat_netirridemand_2020.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [16]:
###2021-NET_IRRIGATION##2021-NET_IRRIGATION###2021-NET_IRRIGATION###2021-NET_IRRIGATION###2021-NET_IRRIGATION###2021-NET_IRRIGATION

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2021/01/01'
sim_end = '2021/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2021"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    #irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatNetIrriDeamad/wheat_netirridemand_2021.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [17]:
###2022-NET_IRRIGATION##2022-NET_IRRIGATION###2022-NET_IRRIGATION###2022-NET_IRRIGATION###2022-NET_IRRIGATION###2022-NET_IRRIGATION

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2022/01/01'
sim_end = '2022/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2022"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    #irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatNetIrriDeamad/wheat_netirridemand_2022.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula

In [18]:
###2023-NET_IRRIGATION##2023-NET_IRRIGATION###2023-NET_IRRIGATION###2023-NET_IRRIGATION###2023-NET_IRRIGATION###2023-NET_IRRIGATION

import pandas as pd
import numpy as np


# Load the data
df = pd.read_csv("daymet_data_with_et0.csv")


# Get a list of unique sites
unique_sites = df['site'].unique()

# Specify the directories where the weather files and simulation results will be saved
weather_dir = "./ClimateData"
sim_output_dir = "./SimulationResults"

# Create the directories if they do not exist
os.makedirs(weather_dir, exist_ok=True)
os.makedirs(sim_output_dir, exist_ok=True)

# Define simulation start and end dates
sim_start = '2023/01/01'
sim_end = '2023/12/30'

# Loop through each unique site
for site in unique_sites:
    # Filter for the current site and year, select specific columns, and rename them
    df_filtered = (df[df['site'] == site]
                   .loc[:, ['Day', 'Month', 'Year', 'MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET']]
                   .rename(columns={
                       'MinTemp': 'Tmin(c)',
                       'MaxTemp': 'Tmax(c)',
                       'Precipitation': 'Prcp(mm)',
                       'ReferenceET': 'Et0(mm)'
                   })
                   .query("Year == 2023"))

    # Check the columns in df_filtered
    print(f"Columns for site {site}: {df_filtered.columns.tolist()}")

    # Check if 'Prcp(mm)' is in the DataFrame
    if 'Prcp(mm)' not in df_filtered.columns:
        print(f"'Prcp(mm)' not found for site {site}. Skipping this site.")
        continue

    # Calculate total precipitation during the growing period (May 1 to October 30)
    total_precip = df_filtered[(df_filtered['Month'] >= 5) & (df_filtered['Month'] <= 10) &
                               ((df_filtered['Month'] != 10) | (df_filtered['Day'] <= 30))]['Prcp(mm)'].sum()

    # Create a filename for the current site's weather data
    weather_filename = os.path.join(weather_dir, f"site_{site}_weather_data.txt")

    # Write the filtered DataFrame to a text file for weather data
    df_filtered.to_csv(weather_filename, sep="\t", index=False, header=True)

    print(f"Weather data for site {site} saved successfully as {weather_filename}.")

    # Prepare weather data for the current site
    wdf = prepare_weather(weather_filename)

    # Run AquaCrop simulations
    irr_mngt = IrrigationManagement(irrigation_method=4,NetIrrSMT=70) # specify irrigation management
    #irr_mngt = IrrigationManagement(irrigation_method=0)  # Specify irrigation management (rainfed in this case)

    # Define the soil and crop instances
    soil = Soil('LoamySand')
    crop = Crop('Wheat',
                planting_date='04/01',
                harvest_date='11/30',
                #CalendarType=2,
                #SwitchGDD=1,
                CropType=3,
                Tbase=10,
                Tupp=35,
                #Maturity=120,
                #PlantPop=2000000,
                #Emergence=42,
                #MaxRooting=597,
                #Flowering=114,
                #Senescence=821,
                #HIstart=224,
                Zmax=0.7,
                WP=16,
                Tmin_up=8,
                Tmax_lo=40,
                exc=50,
                CGC=0.16764,
                CCx=0.95,
                CDC=0.13653,
                #SeedSize=5,
                Kcb=1.10,
                fshape_r=15,
                SxTopQ=0.020,
                SxBotQ=0.005,
                p_up4=0.8,
                p_up2=0.55,
                fshape_w1=4
               )
    initWC = InitialWaterContent(value=['FC'])

    # Create the AquaCrop model instance
    model = AquaCropModel(
        sim_start,
        sim_end,
        wdf,
        soil,
        crop,
        initial_water_content=initWC,
        irrigation_management=irr_mngt
    )

    # Run the model simulation
    try:
        model.run_model(till_termination=True)

        # Create a subdirectory for each site in the simulation output directory
        site_sim_output_dir = os.path.join(sim_output_dir, f"site_{site}")
        os.makedirs(site_sim_output_dir, exist_ok=True)

        # Save the simulation results for the current site
        site_results_filename = os.path.join(site_sim_output_dir, f"site_{site}_simulation_results.csv")
        results_df = model._outputs.final_stats

        # Add a column for total precipitation during the growing period
        results_df['Total_Precipitation(mm)'] = total_precip

        results_df.to_csv(site_results_filename, index=False)

        print(f"Simulation results for site {site} saved successfully in {site_sim_output_dir}.")
    except Exception as e:
        print(f"An error occurred during the simulation for site {site}: {e}")

print("All simulations completed.")

# Merging results code
import pandas as pd
import os

# Directory where simulation results are saved
sim_output_dir = "./SimulationResults"

# Initialize a list to store all merged simulation results
all_simulation_results = []

# Loop through all subdirectories in the SimulationResults directory
for site_dir in os.listdir(sim_output_dir):
    site_path = os.path.join(sim_output_dir, site_dir)

    # Ensure we're only looking at directories
    if os.path.isdir(site_path):
        # Extract the site number from the directory name
        site = site_dir.split('_')[1]

        # Find the CSV file in the current site's directory
        for file in os.listdir(site_path):
            if file.endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(site_path, file)

                # Load the CSV file into a DataFrame
                df = pd.read_csv(file_path)

                # Add a column to identify the site
                df['Site'] = site

                # Append the DataFrame to the list
                all_simulation_results.append(df)

# Combine all DataFrames into one
merged_results = pd.concat(all_simulation_results, ignore_index=True)

output_path = r"/Users/tharakajayalath/Library/CloudStorage/OneDrive-UniversityofSaskatchewan/Chapter II-IrrigationValue/Chapter-II/AquaCropOPSyData/WheatNetIrriDeamad/wheat_netirridemand_2023.csv"

merged_results.to_csv(output_path, index=False)


Columns for site 1: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 1 saved successfully as ./ClimateData/site_1_weather_data.txt.
Simulation results for site 1 saved successfully in ./SimulationResults/site_1.
Columns for site 2: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 2 saved successfully as ./ClimateData/site_2_weather_data.txt.
Simulation results for site 2 saved successfully in ./SimulationResults/site_2.
Columns for site 3: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 3 saved successfully as ./ClimateData/site_3_weather_data.txt.
Simulation results for site 3 saved successfully in ./SimulationResults/site_3.
Columns for site 4: ['Day', 'Month', 'Year', 'Tmin(c)', 'Tmax(c)', 'Prcp(mm)', 'Et0(mm)']
Weather data for site 4 saved successfully as ./ClimateData/site_4_weather_data.txt.
Simulation results for site 4 saved successfully in ./Simula